# 21장 Ajax와 코멧

> 이 장에서 다루는 내용
- XMLHttpRequest 객체의 사용법
- XMLHttpRequest 이벤트의 사용법
- 크로스 도메인 Ajax 제한


- 2005년 제시 제임스 개릿 'Ajax: 웹 애플리케이션의 새로운 접근법' 발표
    - '비동기적 javascript와 XML'을 줄인 말인 'Ajax' 테크닉의 개요가 나와있음
    - 현재 웹 페이지를 떠나지 않은 채 서버에 데이터를 요청하여 사용자의 경험을 더 좋게함
- Ajax의 핵심 기술
    - XMLHttpRequest(XHR) 객체
    - 마이크로소프트에서 처음 개발했으며 다른 브라우저 제조사들도 도입
- Ajax란 이름에 XML이 들어가긴 하지만, 형식과 관계는 없음
- 페이지를 새로고침하지 않고 서버에서 데이터를 받아오는 테크닉일 뿐

## 21.1 XMLHttpRequest 객체
- IE5에서 처음 도임
- MSXML 라이브러리의 일부로 포함된 ActiveX 객체를 통해 XHR 지원

```javascript
function createXHR(){
    if (typeof arguments.callee.activeXString != "string"){
        var versions = ["MSXML2.XMLHttp.6.0", "MSXML2.XMLHttp.3.0", "MSXML2.XMLHttp"],
            i, len;

        for (i=0,len=versions.length; i < len; i++){
            try {
                new ActiveXObject(versions[i]);
                arguments.callee.activeXString = versions[i];
                break;
            } catch (ex){
                //skip
            }
        }
    }
    
    return new ActiveXObject(arguments.callee.activeXString);
}
```

- 인터넷 익스플로러에서 사용 가능한 최신 버전의 XHR 객체 생성
- 인터넷 익스플로러 7 이상, 파이어폭스, 오페라, 크롬, 사파리
    - `var xhr = new XMLHttpRequest();`

### 21.1.1 XHR 사용법
- 먼저 open() 메서드 호출
    - 매개변수
        - 요청타입('get' or 'post')
        - 요청 url
        - 요청을 비동기적으로 보낼지 나타내는 불리언 값
        
```javascript
xhr.open('get', 'example.php', false);
```

- 주의해야 될 점
    - URL은 요청을 보내는 페이지에 대한 상대 경로(절대 경로도 가능)
    - open() 자체는 요청을 보내는 것이 아니라 단순히 요청을 보낼 준비만 하는 것

```javascript
xhr.open('get', 'example.txt', false);
xhr.send(null);
```

- 이 요청은 동기적이므로 응답을 받을 때까지 실행을 멈추고 대기 한다.
- 응답을 받으면 XHR 객체의 프로퍼티에 데이터가 채워진다.
    - 관련 프로퍼티
        - responseText
            - 응답에 포함된 텍스트
        - reponseXML
            - 응답의 콘텐츠 타입이 'text/xml'이나 'application/xml'인 경우 XML DOM 문서 형태의 응답데이터
        - status
            - 응답의 HTTP 상태
        - statusText
            - HTTP 상태에 대한 설명
- 응답을 받으면 먼저 status 프로퍼티를 체크하여 응답이 성공적으로 반환되었는지 확인
    - 일반적으로 200대 숫자이면 성공이며 responseText에 데이터가 있다고 간주해도 된다.
    - 상태코드가 304라면 요청한 자원이 바뀌지 않았으므로 브라우저의 캐시에서 가져왔다는 뜻
    
```javascript
var xhr = createXHR();        
xhr.open("get", "example.txt", false);
xhr.send(null);

if ((xhr.status >= 200 && xhr.status < 300) || xhr.status == 304){
    alert(xhr.statusText);
    alert(xhr.responseText);
} else {
    alert("Request was unsuccessful: " + xhr.status);
}
```

- statusText는 브라우저에 따라 다르므로 항상 status 프로퍼티를 기준으로 판단하길 권한다


- 이 코드처럼 동기적인 요청을 보낼 수도 있지만, 응답을 기다리지 않고 실행을 계속할 수 있는 비동기적 요청이 훨씬 많이 쓰인다
- XHR객체에는 현재의 요청/응답 단계를 나타내는 readyState 프로퍼티가 있다.
    - 0
        - 초기화 전. open() 메서드를 호출하기 전
    - 1
        - 열림. open() 메서드를 호출했지만, send()는 호출하지 않은 상태
    - 2
        - 보냄. send() 메서드를 호출했지만, 응답은 받지 못한 상태
    - 3
        - 수신 중. 응답 데이터 일부를 받았다
    - 4
        - 완료. 응답 데이터를 모두 받았다.
- readyState 값이 바뀔 때마다 readystatechange 이벤트가 발생
- open() 메서드를 호출하기 전에, onreadysatetchange 이벤트 핸들러를 등록해야 한다.

```javascript
var xhr = createXHR();        
xhr.onreadystatechange = function(event){
    if (xhr.readyState == 4){
        if ((xhr.status >= 200 && xhr.status < 300) || xhr.status == 304){
            alert(xhr.responseText);
        } else {
            alert("Request was unsuccessful: " + xhr.status);
        }
    }
};
xhr.open("get", "example.txt", true);
xhr.send(null);
```

- 응답을 받기 전에 다음과 같이 abort() 메서드를 호출하여 비동기적 요청을 취소 할 수 있다.

```javascript
xhr.abort();
```

- 이 메서드를 호출하면, XHR 객체에서 이벤트 발생을 멈추며, 응답과 관련된 프로퍼티에 접근할 수 없게 된다.
- 요청을 취소하면 XHR 객체의 참조를 해제해야 한다.
- 메모리 문제가 있어서 XHR 객체의 재사용은 권장하지 않는다.

### 21.1.2 HTTP 헤더
- 모든 HTTP 요청과 응답에는 헤더 정보가 첨부된다.
- 이 정보 중에는 개발자에게 필요한 것도 있고, 그렇지 않은 것도 있다.
- XHR 객체는 여러 가지 방법으로 요청과 응답의 헤더 정보를 노출한다.
- 기본적으로 XHR 요청을 보낼 때는 다음 헤더 정보가 포함된다.
    - Accept
        - 브라우저가 처리 가능한 콘텐츠 타입
    - Accept-Charset
        - 브라우저가 표시 가능한 문자셋
    - Accept-Encoding
        - 브라우저가 처리 가능한 압축 인코딩
    - Accept-Language
        - 브라우저의 언어
    - Connection
        - 브라우저와 서버의 연결 타입
    - Cookie
        - 페이지에 설정된 쿠키
    - Host
        - 요청을 보내는 페이지의 도메인
    - Referer
        - 요청을 보내는 페이지의 URI, 이 헤더 이름은 HTTP 명세의 오타 였지만, 호환성을 위해 반드시 이대로 써야된다.(정확한 단어는 referrer)
    - User-Agent
        - 브라우저의 사용자 에이전트 문자열
- setRqquestHeader() 메서드를 써서 요청 헤더를 추가할 수 있다.
    - 반드시 open()과 send() 사이에서 호출해야 한다.
- XHR 객체의 응답 헤더를 가져오려면 getResponseHeader() 메서드에 헤더 이름을 넘겨 호출한다.
- getAllResponseHeaders() 메서드를 쓰면 헤더 전체를 긴 문자열 형태로 반환한다.

```javascript
var myHeader = xhr.getResponseHeader('MyHeader');
var allHeaders = xhr.getAllResponseHeaders();
```

### 21.1.3 GET 요청
- GET 요청은 일반적으로 서버에 특정 정보를 쿼리할 때 사용하는 것으로 가장 많이 쓰이는 요청
- URL 마지막에 쿼리스트링 매개변수를 추가해서 서버에 요청할 수 있다.
- 반드시 정확히 인코드해서 open() 메서드에 넘기는 URL에 연결해야 한다.
- 가장 흔한 에러는 쿼리스트링 형식이 잘못된 경우이다
- 쿼리스트링의 각 이름과 값은 반드시 encodeURIComponent()로 인코드해서 URL에 첨부해야 하며 반드시 다음과 같이 앰퍼샌드로 구분해야 한다.

```javascript
xhr.open('get', 'example.php?name1=value1&name2=value2', true);
```

### 21.1.4 POST 요청
- POST 요청은 일반적으로 서버에서 저장할 데이터를 보낼 때 쓰인다.
- GET 요청과 달리 대개 본문에 데이터가 존재한다.
- POST 요청 본문에는 매우 많은 데이터를 쓸 수 있으며, 데이터 형식에도 제한이 없다.

## 21.2 XMLHttpRequest 레벨 2
- XHR 객체가 사실상으로 표준으로 인기를 얻자 W3C에서 이의 동작을 정의하는 공식적인 명세를 만듦
- XMLHttpRequest 레벨 1은 단순히 이미 존재하는 XHR 객체를 정의
- XMLHttpRequest 레벨 2에서는 XHR 객체를 더 발전시킴

### 21.2.1 FormData 타입
- 최신 웹 애플리케이션에서는 폼 데이터를 직렬화하는 일이 많으므로 XMLHttpRequest 레벨 2 명세에서는 FormData 타입을 도입
- FormData 타입을 사용하면
    - 이미 존재하는 폼을 직렬화하기도 쉽고
    - 데이터를 폼 같은 형식으로 생성하여 XHR로 전송하기도 쉽다.

```javascript
var data = new FormData();
data.append('name', 'nicholas');
```

- 다음과 같이 FormData인스턴스를 직접 XHR send()메서드에 전달할 수 있다.

```javascript
var xhr = createXHR();        
xhr.onreadystatechange = function(event){
    if (xhr.readyState == 4){
        if ((xhr.status >= 200 && xhr.status < 300) || xhr.status == 304){
            alert(xhr.responseText);
        } else {
            alert("Request was unsuccessful: " + xhr.status);
        }
    }
};

xhr.open("post", "postexample.php", true);
var form = document.getElementById("user-info");            
xhr.send(new FormData(form));
```

### 21.2.2 타임아웃
- 인터넷 익스플로러 9에서는 XHR 객체에 timeout 프로퍼티를 추가했다.
- 이 프로퍼티는 요청을 취소하기 전에 몇 밀리초 동안 응답을 기다릴지 나타낸다.
- timeout 프로퍼티를 숫자로 지정하고, 그 시간 동안 응답이 없으면 timeout 이벤트가 발생하고, ontimeout 이벤트 핸들러가 호출
- 나중에 XMLHttpRequest 레벨 2 명세에 추가되었다.

```javascript
var xhr = createXHR();        
xhr.onreadystatechange = function(event){
    try {
        if (xhr.readyState == 4){
            if ((xhr.status >= 200 && xhr.status < 300) || xhr.status == 304){
                alert(xhr.responseText);
            } else {
                alert("Request was unsuccessful: " + xhr.status);
            }
        }
    } catch (ex){
        // ontimeout에서 처리한 것으로 간주
    }
};

xhr.open("get", "timeout.php", true);
xhr.timeout = 1000; // 타임아웃을 1초로 설정
xhr.ontimeout = function(){
    alert("Request did not return in a second.");
};        
xhr.send(null);
```

### 21.2.3 overrideMimeType() 메서드
- 파이어폭스는 overrideMimeType()를 도입하여 XHR 응답의 마임 타입을 덮어 쓸 수 있게 했다.
- 이 메서드는 XMLHttpRequest 레벨 2 명세에 추가되었다.
- XML 객체는 응답을 처리할 때, 반환받은 마임 타입에 따라 분기하므로 응답 서버에서 변환한 타입을 덮어쓸 수 있다면 유용할 때가 많다.
- 서버에서 반환한 마임타입이 text/plain 이지만 사실은 XML이라고 가정
- 데이터가 XML 이지만 responseXML 프로퍼티는 null 일 것이다.
- 다음과 같이 overrideMymeType()을 사용하면 응답을 텍스트가 아니라 XML로 처리 한다.

```javascript
var xhr = createXHR();
xhr.open('get', 'text.php', true);
xhr.overrideMimeType('text/xml');
xhr.send(null);
```

- 이 예제는 XHR 객체에서 응답을 XML로 간주하고 처리하도록 강제
- 반드시 overrideMimeType()을 send()보다 먼저 호출해야 한다.
- [MIME 타입](https://developer.mozilla.org/ko/docs/Web/HTTP/Basics_of_HTTP/MIME_types)

## 21.3 진행상태 이벤트
- 진행상태<sup>progress</sup> 이벤트 명세는 클라이언트-서버 통신 이벤트를 정의한 W3C 초안
- 원래 XHR 마을 염두에 두고 만들어졌지만, 최근에는 다른 비슷한 API에서도 사용
- 진행상태 이벤트는 6가지
    - loadstart
        - 응답의 첫 번째 바이트를 수신했을 때 발생
    - progress
        - 응답을 받는 동안 계속 발생
    - error
        - 요청에 에러가 있을 때 발생
    - abort
        - abort()를 호출해 연결을 끊었을 때 발생
    - load
        - 응답을 완전히 받았을 때 발생
    - loadend
        - 통신이 끝났을 때 발생하며 error, abort, load보다 나중에 발생
- 각 요청은 loadstart 이벤트로 시작, 그 다음에 progress 이벤트가 여러 번 발생하며, error와 abort, load 중 하나가 뒤를 잇고, 마지막으로 loadend가 발생
- 현재 loadend 이벤트를 지원하는 브라우저는 없다(?)

### 21.3.1 load 이벤트
- 파이어폭스는 XHR 객체를 처음 구현했을 때, 상호작용 모델을 단순화하려고 load이벤트를 도입하면서 readystatechange 이벤트를 대체
- load 이벤트는 응답을 완전히 받는 순간 발생하므로 readystate 프로퍼티를 확인할 필요가 없어진다.
- onload 이벤트 핸들러가 받는 event 객체의 target 프로퍼티는 XHR 객체인스턴스이며, 이를 통해 XHR 객체의 멤버 전체에 접근가능하다.
- 하지만 모든 브라우저에서 이 이벤트에 event 객체를 제대로 구현하진 않았으므로 다음 예제처럼 XHR 객체 변수를 써야 한다.

```javascript
var xhr = createXHR();        
xhr.onload = function(event){
    if ((xhr.status >= 200 && xhr.status < 300) || xhr.status == 304){
        alert(xhr.responseText);
    } else {
        alert("Request was unsuccessful: " + xhr.status);
    }
};
xhr.open("get", "altevents.php", true);
xhr.send(null);
```

- 서버에서 응답을 받기만 하면 그 상태와 무관하게 load 이벤트 발생
- 따라서 필요한 데이터가 모두 있는지 알아보려면 반드시 status 프로퍼티를 확인해야 한다.
- load 이벤트 지원 브라우저
    - 파이어폭스, 오페라, 크롬, 사파리
    
### 21.3.2 progress 이벤트
- 브라우저가 새 데이터를 받는 동안 주기적으로 발생
- onprogress 이벤트 리스너가 받는 event 객체의 target 프로퍼티는 XHR 객체이며, 세 가지 프로퍼티가 추가
    - lengthComputable
        - 진행상태 정보를 알 수 있는지 나타내는 불리언
    - position
        - 이미 수신한 바이트
    - totalSize
        - Content-Length 응답 헤더에서 가져온 예상 바이트 숫자
- 이런 정보가 있으면 사용자에게 진행상태를 알릴 수 있다.

```javascript
var xhr = createXHR();        
xhr.onload = function(event){
    if ((xhr.status >= 200 && xhr.status < 300) || xhr.status == 304){
        alert(xhr.responseText);
    } else {
        alert("Request was unsuccessful: " + xhr.status);
    }
};
xhr.onprogress = function(event){
    var divStatus = document.getElementById("status");
    if (event.lengthComputable){
        divStatus.innerHTML = "Received " + event.position + " of " + event.totalSize + " bytes";
    }
};
xhr.open("get", "altevents.php", true);
xhr.send(null);
```

## 21.4 소스 간 자원 공유
- XHR을 사용한 Ajax 통신의 주요 제한은 동일 소스 보안 정책
- 기본적으로 XHR 객체는 웹 페이지와 같은 도메인에 있는 자원만 가져올 수 있다.
- 이 보안기능은 일부 위험하고 악의적인 동작을 막는 긍정적인 효과가 있다.
- 하지만 정당한 목적으로 다른 소스의 자원에 접근하는 일도 많아서 이를 위한 해결책이 개발되었다.
- 소스 간 자원 공유<sup>Cross-Origin Resource Sharing: CORS</sup> W3C 초안은 브라우저가 다른 서버와 통신할 때 반드시 지켜야 하는 방식을 정의한 명세
- CORS 기본 아이디어는 브라우저와 서버 쌍방이 이해하는 커스텀 HTTP 헤더를 삽입해서 이를 근거로 요청이나 응답에 반응할지 결정하자는 것
- GET이나 POST를 쓰고 요청 본문이 text/plain이며 커스텀 헤더가 없는 단순한 요청에는 Origin이라는 헤더가 추가된다.
- Origin 헤더에는 요청을 보내는 페이지의 소스(프로토콜, 도메인 이름, 포트)가 들어 있으므로 서버는 이를 근거로 응답할지 하지 않을지를 쉽게 판단 가능하다.
- 서버가 요청에 응답하기로 결정하면 Access-Control-Allow-Origin 헤더를 반송하는, 이는 Origin과 같은 소스일 수도 있고, 공개된 자원이라면 `"*"` 일 수도 있다.
- 이 헤더가 없거나 소스가 맞지 않으면 브라우저에서 요청을 거부한다.
- 헤더가 있고 소스도 일치하면 브라우저에서 요청을 처리한다.
- 참고자료
    - https://developer.mozilla.org/ko/docs/Web/HTTP/CORS
    - https://brownbears.tistory.com/336
    - [CORS, Preflight, 인증 처리 관련 삽질](https://www.popit.kr/cors-preflight-%EC%9D%B8%EC%A6%9D-%EC%B2%98%EB%A6%AC-%EA%B4%80%EB%A0%A8-%EC%82%BD%EC%A7%88/)
    
### 21.4.1 인터넷 익스플로러의 CORS
- 마이크로소프트는 인터넷 익스플로러 8에서 XDomainRequest(XDR) 타입을 도입
- 이 객체는 XHR과 비슷하게 동작하지만 소스 간 통신에도 안전한 방법을 사용
- XDR 객체는 CORS 명세 일부를 구현
- XDR과 XHR의 차이
    - 요청이나 응답에 쿠키가 포함되지 않는다.
    - Content-Type 외에 다른 요청 헤더는 수정할 수 없다.
    - 응답 헤더에는 접근할 수 없다.
    - GET, POST 요청만 지원한다.
- 이러한 변경은 '교차 사이트 요청 위조(Cross-Site Request Forgery: CSRF)'나 교차 사이트 스크립트(XSS) 공격과 관련된 문제를 해결하기 위한 것

### 21.4.2 다른 브라우저의 CORS
- 파이어폭스 3.5 이상, 사파리 4 이상, 크롬, iOS용 사파리, 안드로이드 웹킷은 모두 XMLHttpRequest 객체 자체에서 CORS를 지원
- 다른 소스에 있는 자원에 접근하면 자동으로 필요한 동작이 이루어지므로 다른 코드를 추가할 필요는 없다.
- 다른 도메인에 있는 자원을 요청할 때는 XHR 객체를 그대로 쓰면서 다음과 같이 open()에 절대 URL을 넘기기만 하면 된다.

```javascript
var xhr = createXHR();        
xhr.onreadystatechange = function(event){
    if (xhr.readyState == 4) {
        if ((xhr.status >= 200 && xhr.status < 300) || xhr.status == 304){
            alert(xhr.responseText);
        } else {
            alert("Request was unsuccessful: " + xhr.status);
        }        
    }
};
xhr.open("get", "http://www.somewhere-else.com/page/", true);
xhr.send(null);
```

- 인터넷 익스플로러의 XDR 객체와는 달리 크로스 도메인 XHR 객체는 status와 statusText프로퍼티에 접근 가능하고 동기적인 요청도 허용
- 필요한 보안 목적으로 크로스 도메인 XHR 객체에 추가된 제한
    - setRequestHeader()로 커스텀 헤더를 추가할 수 없다.
    - 쿠키는 송수신 되지 않는다.
    - getAllResponseHeaders() 메서드는 항상 빈 문자열을 반환한다.
- 요청이 크로스 도메인이든 아니든 같은 인터페이스를 사용하므로 로컬 자원에 접근할 때는 항상 상대 URL을 쓰고, 원격 자원에는 절대 URL을 쓰길 권장
- 이렇게 하면 모호함이 사라지고 로컬 자원에서 헤더나 쿠키를 쓸 수 없는 문제가 예방된다.

### 21.4.3 사전 요청
- CORS는 서버의 '사전 요청<sup>preflighted request</sup>'이란 메커니즘을 통해 커스텀 헤더, GET/POST 이외의 요청 타입, 본문 컨텐츠 타입을 허용
- 이런 고급 요청을 보낼 때는 '사전' 요청을 사용
- 이 요청은 OPTIONS 타입을 사용하며 다음 헤더 정보를 포함한다.
    - Origin
        - 단순한 요청과 같다.
    - Access-Control-Request-Method
        - 요청에서 사용하려는 타입
    - Access-Control-Reuqest-Headers
        - (옵션) 쉼표로 구분된 커스텀 헤더 목록

### 21.4.4 인증된 요청
- 크로스 도메인 요청에는 기본적으로 쿠키나 HTTP인증, 클라이언트 SSL인증 같은 인증을 제공하지 않는다.
- 요청에서 반드시 인증을 보내야 한다고 설정하려면 withCredentials 프로퍼티를 true로 지정
- 서버에서 인증된 요청을 수락하면 응답에 다음 HTTP 헤더를 첨부한다.

```ini
Access-Control-Allow-Credentials: true
```

- 인증된 요청을 보냈지만 응답에 이 헤더가 포함되지 않았다면 브라우저는 응답을 자바스크립트에 전달하지 않는다.
- responseText는 빈 문자열이고 status는 0이며 onerror()를 호출
- 서버에서 인증된 요청을 수락하면 사전 응답에 이 헤더를 첨부한다.
- 파이어폭스 3.5 이상, 사파리 4 이상, 크롬에서 withCredentials 프로퍼티를 지원

## 21.5 기타 크로스 도메인 테크닉
- CORS가 쓰이기 전에 크로스 도메인 Ajax 통신을 사용하려면 다소 속임수같은 방법을 써야 했다.
- 개발자들은 DOM요소를 이용해 XHR 객체를 쓰지 않고, 크로스 도메인 요청을 보내는 방법 고안
- CORS는 어디서든 쓸 수 있지만, 이들 테크닉은 서버 설정을 바꿀 필요가 없는 장점이 있어 아직 널리 쓰인다.

### 21.5.1 이미지 핑
- `<img>` 태그를 통해서 크로스 도메인 통신 가능
- 이미지를 불러올 때는 크로스 도메인 제한이 전혀 적용되지 않는다.
- 온라인 광고주들이 방문자를 기록할 때 주로 사용하는 방법
- 동적으로 이미지를 생성하고, onload와 onerror 이벤트를 통해 응답이 오는 시점을 알 수 있다.

```javascript
var img = new Image();
img.onload = img.onerror = function(){
    alert("Done!");
};
img.src = "http://www.example.com/test?name=Nicholas"; 
```

- 보통 사용자의 클릭을 기록하거나 동적 광고에 사용

#### JSONP
- '패딩을 댄 JSON(JSON with padding)'의 약자이며, 웹 서비스에 널리 쓰이는 JSON의 변형
- JSONP는 JSON과 거의 같은데 데이터가 함수 호출과 거의 비슷한 래퍼로 감싸여 있다는 점만 다르다

```javascript
callback({'name': 'nicholas'});
```

- JSONP형식은 콜백과 데이터 두 가지로 구성
    - 콜백
        - 응답을 받은 페이지에서 실행할 함수
        - 일반적으로 콜백 이름은 요청에 포함
    - 데이터
        - 콜백 함수에 전달할 JSON 데이터
- JSONP는 동적 `<script>`요소의 src 속성을 크로스 도메인 URL로 설정하여 사용
- `<script>`요소는 `<img>`와 비슷하게 다른 도메인의 자원을 제한 없이 불러올 수 있다.
- JSONP도 유효한 자바스크립트이므로 JSONP응답을 페이지에 삽입하면 즉시 실행된다.

```javascript
function handleResponse(response){
    alert("You're at IP address " + response.ip + ", which is in " + response.city + ", " + response.region_name);
}

var script = document.createElement("script");
script.src = "http://freegeoip.net/json/?callback=handleResponse";
document.body.insertBefore(script, document.body.firstChild);
```

- 단순하고 사용하기 쉬워서 매우 인기가 있다.
- 이미지 핑에 비해 응답 텍스트에 직접 접근할 수 있어 양방향 통신이 가능하다는 장점
- JSONP의 단점
    - 다른 도메인에서 가져온 코드를 페이지에 직접 삽입
        - 신뢰할 수 없는 도메인에서는 매우 쉽게 응답을 악의적인 코드로 바꿔칠 수 있다.
    - JSONP요청이 실패 했을 때, 이를 알기 어렵다.
    
### 21.5.2 코멧
- 알렉스 러셀이 만든 용어
- 서버 푸시라고 부르기도 하는 고급 ajax 테크닉
- ajax
    - 사용자의 페이지에서 데이터를 요청하는 입장
- 코멧
    - 서버가 사용자 페이지에 데이터를 전달하는 방식
- 거의 실시간으로 페이지에 전달, 스포츠 점수나 주식시장 가격 같은 정보 전달하기에 이상적
- '롱 폴링<sup>long polling</sup>', '스트리밍<sup>streaming</sup>' 방법이 주로 쓰임
- 롱 폴링
    - 브라우저에서 서버로 일정 시간마다 요청을 보내 받아올 데이터가 있는지 알아보는 기존의 폴링방식('숏 폴링')을 크게 바꿨다.
    - 마치 숏 폴링을 접은 것 처럼 동작
    - 페이지에서 서버에 요청을 보내면 서버는 전송할 데이터가 생길 때까지 해당 연결을 계속 유지
    - 데이터를 보내면 브라우저에서 해당 연결을 끊고 즉시 새 연결을 만듦
    
![img/21_1.jpg](img/21_1.jpg)
    
- HTTP 스트리밍
    - 페이지가 존재하는 시간 동안 단 하나의 HTTP연결만 사용
    - 브라우저가 서버에 요청을 보내면 서버는 연결을 열고 주기적으로 데이터를 보낸다.
    
```php
<?php
    $i = 0;
    while(true) {
    
        // 데이터를 출력한 후 버퍼를 즉시 비움
        echo "Number is $i";
        flush();

        // 1초 동안 기다림
        sleep(10);

        $i++;
    }
?>
```

- 서버사이드 언어는 모두 출력 버퍼에 데이터를 모았다가 한 번에 클라이언트로 보내는 방식을 지원, HTTP 스트리밍의 핵심

```javascript
function createStreamingClient(url, progress, finished){        
    var xhr = new XMLHttpRequest(),
        received = 0;

    xhr.open("get", url, true);
    xhr.onreadystatechange = function(){
        var result;

        if (xhr.readyState == 3){

            // 새 데이터만 가져오고 카운터를 업데이트
            result = xhr.responseText.substring(received);
            received += result.length;

            // progress 콜백을 호출
            progress(result);

        } else if (xhr.readyState == 4){
            finished(xhr.responseText);
        }
    };
    xhr.send(null);
    return xhr;
}

var client = createStreamingClient("streaming.php", function(data){
                alert("Received: " + data);
             }, function(data){
                alert("Done!");
             });
```

### 21.5.3 서버 송신 이벤트
- '서버 송신 이벤트'(Server-Send Event: SSE)
- 코멧에서 읽기 전용 상호작용을 제공하는 API와 패턴
- SSE API는 서버로 단방향 HTTP연결을 생성하며, 서버는 이 연결을 통해 가능한 한 작은 정보만 보낸다.
- 숏 폴링, 롱 폴링, HTTP스트리밍을 모두 지원
- 연결이 끊어졌을 때 서버에 다시 연결할 타이밍을 자동으로 판단

### 21.5.4 웹소켓
- 오래 지속되는 연결 단 한만 사용하면서 완전한 양방향 통신을 제공
- 자바스크립트에서 웹 소켓을 생성하면 서버로 HTTP요청을 보내 연결을 초기화
- 서버에서 응답하면 연결은 HTTP에서 웹 소켓 프로토콜로 업그레이드
- 즉, 표준 HTTP 서버에서는 웹 소켓을 구현할 수 없으며 반드시 서버에서 특별한 프로토콜을 지원

```javascript
var socket = new WebSocket('ws://www.example.com/server.php');
socket.send('hello world!');

socket.close();
```